In [15]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

from models.layers.layers import EctLayer
from models.config import EctConfig

from torch_geometric.data import Batch, Data
import torchmetrics
import matplotlib.pyplot as plt 
from datasets.modelnet import ModelNetDataModule
from datasets.config import ModelNetDataModuleConfig

dataset = ModelNetDataModule(ModelNetDataModuleConfig(root="./data/modelnet_40_100",samplepoints=100))
print(dataset.entire_ds[0])
ECT_SIZE = 128
DEVICE = "cuda:0"

data = dataset.entire_ds[0]
data.x.max()

Data(y=[1], x=[100, 3])


tensor(0.8765)

In [16]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = EctLayer(config = EctConfig(num_thetas=ECT_SIZE,bump_steps=ECT_SIZE,num_features=3,device=DEVICE))

        # encoder_layer = nn.TransformerEncoderLayer(
        #     d_model=64,
        #     nhead=4,
        #     dim_feedforward=256,
        #     dropout=0.0,
        # )
        # self.transformer_encoder = nn.TransformerEncoder(
        #     encoder_layer,
        #     num_layers=6,
        # )
        
        HIDDEN = 128

        self.classifiers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(ECT_SIZE**2,HIDDEN), # (N, 784) -> (N, 128)
            nn.ReLU(),
            nn.Linear(HIDDEN, HIDDEN),
            nn.ReLU(),
            nn.Linear(HIDDEN, 40) 
        )

    def forward(self, batch):
        x = self.layer(batch)
        # x = self.transformer_encoder(x)
        # x = x.mean(dim=1)
        x = self.classifiers(x)
        return x

In [17]:
model = Autoencoder().to(DEVICE)


for batch in dataset.train_dataloader():
    batch = batch.to(DEVICE)
    # img = img.reshape(-1, 28*28) # -> use for Autoencoder_Linear

res = model(batch)
print(res.shape)

torch.Size([2, 40])


In [18]:
def compute_acc(model, loader, num_classes=None):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.eval()
    acc = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes).to(device)
    # auroc = AUROC(task="multiclass", num_classes=num_classes).to(device)
    loss_fn = torch.nn.CrossEntropyLoss()
    loss = torch.tensor([0.0], device=device)
    with torch.no_grad():
        for batch in loader:
            batch_gpu, y_gpu = batch.to(device), batch.y.to(device)
            logits = model(batch_gpu)
            loss += loss_fn(logits, y_gpu)
            # auroc(logits, y_gpu)
            acc(logits, y_gpu)
    a = acc.compute()
    # roc = auroc.compute()
    acc.reset()
    # auroc.reset()
    return loss, a, torch.tensor([0])


In [19]:
model = Autoencoder().to(DEVICE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

In [20]:
for epoch in range(100):
    model.train()
    for batch in dataset.train_dataloader():
        batch_gpu, y_gpu = batch.to(DEVICE), batch.y.to(DEVICE)
        optimizer.zero_grad()
        pred = model(batch_gpu)
        loss = loss_fn(pred, y_gpu)
        loss.backward()
        optimizer.step()
        # raise "hello"

    val_loss, acc, _ = compute_acc(model, dataset.val_dataloader(), 40)

    # del batch_gpu, y_gpu, pred, loss
    print(f'Epoch:{epoch+1}, Loss:{loss.item():.4f}, Acc:{acc.item():.4f}')


Epoch:1, Loss:4.0066, Acc:0.0396
Epoch:2, Loss:3.2945, Acc:0.0528
Epoch:3, Loss:3.4037, Acc:0.0599
Epoch:4, Loss:3.2990, Acc:0.0716
Epoch:5, Loss:3.3504, Acc:0.1112
Epoch:6, Loss:3.1743, Acc:0.1290
Epoch:7, Loss:3.3127, Acc:0.1346
Epoch:8, Loss:3.7926, Acc:0.1712
Epoch:9, Loss:2.7485, Acc:0.2031
Epoch:10, Loss:2.7308, Acc:0.2732
Epoch:11, Loss:2.6822, Acc:0.2936
Epoch:12, Loss:2.7383, Acc:0.2646
Epoch:13, Loss:2.4175, Acc:0.3149
Epoch:14, Loss:2.8897, Acc:0.2753
Epoch:15, Loss:2.9012, Acc:0.3261
Epoch:16, Loss:2.2556, Acc:0.2854
Epoch:17, Loss:1.0269, Acc:0.3738
Epoch:18, Loss:2.3719, Acc:0.3266
Epoch:19, Loss:1.6876, Acc:0.3443
Epoch:20, Loss:2.3703, Acc:0.4053
Epoch:21, Loss:3.2744, Acc:0.3956
Epoch:22, Loss:1.1847, Acc:0.4099
Epoch:23, Loss:1.4998, Acc:0.4266
Epoch:24, Loss:1.9759, Acc:0.3677
Epoch:25, Loss:6.4440, Acc:0.3337
Epoch:26, Loss:3.8111, Acc:0.4033
Epoch:27, Loss:2.9135, Acc:0.3804
Epoch:28, Loss:2.1379, Acc:0.4342
Epoch:29, Loss:1.3414, Acc:0.4231
Epoch:30, Loss:4.0613, 

In [12]:
val_loss, acc, _ = compute_acc(model, dataset.test_dataloader(), 40)
print(acc)


tensor(0.5028, device='cuda:0')
